CLEAN SKILL

In [56]:
from __future__ import division
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import string
import time
import random
import os

# BASE_PATH = os.path.dirname(__file__)

class CleanSkills():

    def __init__(self, input_skill_file,  clean_skill_file, output_skill_file):
        if type(input_skill_file) is pd.core.frame.DataFrame:
            self.input_skills_list = input_skill_file['lemmatised_skill']
        else:
            self.input_skills_list = pd.read_csv(input_skill_file)['lemmatised_skill']

        if type(clean_skill_file) is pd.core.frame.DataFrame:
            self.clean_skills_list = clean_skill_file['skill']
        else:
            self.clean_skills_list = pd.read_csv(clean_skill_file)['skill']
        self.output_clean_skill_file = output_skill_file
        self.res = {}        

    def clean_skills(self, output_type='DataFrame'):
        result_df = pd.DataFrame()
        cols = ['skill', 'clean_skill', 'actual_skill']

        # self.input_skills_list = ['oracle database: 11g', 'self confidant', 'uv visible spectroscopy', 'junior software developer', 'angular material design', 'signal processsing']
        for skill in self.input_skills_list:
            skill = str(skill).strip()
            if skill.lower() in self.res.keys():
                skills_stat_df = pd.DataFrame({'skill':[skill], 'clean_skill':[self.res[skill.lower()]]})
                result_df = result_df.append(skills_stat_df)
                continue
            actual_skill, skill_list, clean_skill_list, total_score_list, ratio_list, partial_ratio_list,\
            sort_ratio_list, partial_sort_ratio_list, set_ratio_list, partial_set_ratio_list = [], [], [], [], [], [], [], [], [], []
            for clean_skill in self.clean_skills_list:
                clean_skill = str(clean_skill)
                actual_skill.append(clean_skill)
                clean_skill = clean_skill.lower()
                ratio = fuzz.ratio(skill, clean_skill)
                partial_ratio = fuzz.partial_ratio(skill, clean_skill)
                sort_ratio = fuzz.token_sort_ratio(skill, clean_skill)
                partial_sort_ratio = fuzz.partial_token_sort_ratio(skill, clean_skill)
                set_ratio = fuzz.token_set_ratio(skill, clean_skill)
                partial_set_ratio = fuzz.partial_token_set_ratio(skill, clean_skill)

                total_score = (1 * ratio) + (0.8 * set_ratio) + (0.8 * sort_ratio) + (0.5 * partial_ratio) + \
                              (0.3 * partial_set_ratio) + (0.3 * partial_sort_ratio)
                
                skill_list.append(skill)
                clean_skill_list.append(clean_skill)
                total_score_list.append(total_score)
                ratio_list.append(ratio)
                partial_ratio_list.append(partial_ratio)
                sort_ratio_list.append(sort_ratio)
                partial_sort_ratio_list.append(partial_sort_ratio)
                set_ratio_list.append(set_ratio)
                partial_set_ratio_list.append(partial_set_ratio)
            skills_stat_df = pd.DataFrame({'skill':skill_list, 'clean_skill':clean_skill_list,\
                            'actual_skill':actual_skill,\
                            'total_score':total_score_list, 'ratio':ratio_list,\
                            'partial_ratio':partial_ratio_list, 'sort_ratio':sort_ratio_list,\
                            'partial_sort_ratio':partial_sort_ratio_list, 'set_ratio':set_ratio_list, 'partial_set_ratio':partial_set_ratio_list})\
                            .sort_values(by=['total_score', 'ratio', 'sort_ratio', 'set_ratio', 'partial_ratio'], ascending=False)[:1].reset_index(drop=True)
#             print(skills_stat_df['skill'])
#             print(skills_stat_df['clean_skill'])
            self.res[skills_stat_df['skill'][0].lower()] = skills_stat_df['actual_skill'][0]
            result_df = result_df.append(skills_stat_df[cols])

        if output_type != 'DataFrame':
            result_df.to_csv(self.output_clean_skill_file)
        else:
            return result_df.reset_index(drop=True)


LEMMATISE SKILLS

In [ ]:
# import os
# import nltk
# from nltk.stem import WordNetLemmatizer
# import csv
# import sys
# import string

# class LemmatiseSkills():
    
#     def __init__(self):
#         self.wordnet_lemmatizer = WordNetLemmatizer()
#         # nltk.download('wordnet')

#     def lemmatise_file(self, in_file, out_file):
#         with open(in_file, 'r') as input_file, open(out_file, 'w') as output_file:
#             line = input_file.readline()
#             while line:
#                 flag = 0
#                 for char in line:
#                     if ord(char) > 127:
#                         flag = 1
#                         break
#                 if flag:
#                     output_file.write('" "\n')
#                     line = input_file.readline()
#                     continue
#                 line = line.translate(str.maketrans('-/"()&', '      ', ''))
#                 new_line = ''
#                 for word in line.split(' '):
#                     word = word.strip().lower()
#                     lemma = self.wordnet_lemmatizer.lemmatize(word)
#                     new_line += word+' ' if len(word) <= 4 else lemma+' '
#                 line = input_file.readline()
#                 output_file.write('"{new_line}"\n'.format(new_line=new_line.strip()))


import os
import nltk
from nltk.stem import WordNetLemmatizer
import csv
import sys
import string

class LemmatiseSkills():
    
    def __init__(self):
        self.wordnet_lemmatizer = WordNetLemmatizer()
        self.out_df = pd.DataFrame()
        # nltk.download('wordnet')

    def lemmatise_file(self, in_df, column_name):
        result = []
        for index, row in in_df.iterrows():
            line = str(row[column_name])
            flag = 0
            for char in line:
                if ord(char) > 127:
                    flag = 1
                    break
            if flag:
                result.append(' ')
                continue
            line = line.translate(str.maketrans('-/"()&', '      ', ''))
            new_line = ''
            for word in line.split(' '):
                word = word.strip().lower()
                lemma = self.wordnet_lemmatizer.lemmatize(word)
                new_line += word+' ' if len(word) <= 4 else lemma+' '
            result.append(new_line.strip())
        self.out_df[column_name] = result

CLEANING DRIVER

In [ ]:
verified_skills_file = '/home/jasmeet16-jtg/projects/temp_scripts/skill_standardization/verified_skills.csv'
not_verified_skills_file = '/home/jasmeet16-jtg/projects/temp_scripts/skill_standardization/input_unverified_skills.csv'

verified_skill_all_data = verified_skills = pd.read_csv(verified_skills_file)

vs1 = verified_skills[verified_skills['Status']=='Verified']['Name']
vs2 = verified_skills[verified_skills['New Status']=='Verified']['New Name']
verified_skills = pd.concat([pd.DataFrame({'skill': vs1}), pd.DataFrame({'skill': vs2})])
verified_skills.to_csv('/home/jasmeet16-jtg/projects/temp_scripts/skill_standardization/all_skills_inc_calyx.csv')

vs1 = verified_skill_all_data[verified_skill_all_data['Status']=='Verified']
vs2 = verified_skill_all_data[verified_skill_all_data['New Status']=='Verified']
verified_skill_all_data = pd.concat([vs1, vs2])
verified_skill_all_data.to_csv('/home/jasmeet16-jtg/projects/temp_scripts/skill_standardization/only_verified_skills_data.csv')

not_verified_skills = pd.read_csv(verified_skills_file)
not_verified_skills = not_verified_skills[(not_verified_skills['Status'] != 'Verified')\
                                          & (not_verified_skills['New Status'] != 'Verified')].reset_index(drop=True)

ls = LemmatiseSkills()
ls.lemmatise_file(not_verified_skills, 'Name')

not_verified_skills['lemmatised_skill'] = ls.out_df['Name']
not_verified_skills.to_csv(not_verified_skills_file)

In [ ]:
chunksize = 20
final_output = '/home/jasmeet16-jtg/projects/temp_scripts/skill_standardization/final_output.csv'
verified_skills = pd.read_csv('/home/jasmeet16-jtg/projects/temp_scripts/skill_standardization/all_skills_inc_calyx.csv')
i = 0
header=True
for chunk in pd.read_csv(not_verified_skills_file, chunksize=chunksize):
    chunk = chunk.reset_index(drop=True)
    cs = CleanSkills(chunk, verified_skills, 'xyz')
    df = cs.clean_skills()
    chunk['New Name'] = df['actual_skill']
    chunk['New Status'] = ['Delete']*chunksize
    chunk.to_csv(final_output, mode='a', header=header, index=False)
    i += chunksize
    print('Cleaned {i} skills'.format(i=i))
    header=False


/home/jasmeet16-jtg/projects/virtualenvs/calyx_ds3/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Cleaned 20 skills
Cleaned 40 skills
Cleaned 60 skills
Cleaned 80 skills
Cleaned 100 skills
Cleaned 120 skills
Cleaned 140 skills
Cleaned 160 skills
Cleaned 180 skills


In [52]:
q = pd.DataFrame({'x':[1,2,3,4]})
q = q[(q['x']==1) | (q['x']==4)]
q = q.reset_index(drop=True)
q

,x
0,1
1,4


In [43]:
df['clean_skill']

0                       2g
1                 concepts
2                      k-1
3                   x 10.1
4                      10k
5                      10k
6     personal development
7         ssl certificates
8                    linux
9           plain language
10                database
11                   solar
12                 loyalty
13                      2g
14                  ubuntu
15                    java
16            positive pay
17                   j1939
18               lotus 123
19                 circuit
Name: clean_skill, dtype: object

system admin > data verification > verification scripts